# Part V - The News Hour - Estimation Setup

Now we have in previous notebooks set up the problem, drawn some preliminary values, set up some selection terms, and so on. Now, we can finally estimate the model - after a little bit more work - which will involve writing down a big likelihood function (of which some parts we already have). Before doing this, let's read in our data and get it into mata. 

In [2]:
import ipystata
import os
os.getcwd()

'C:\\Users\\mjbaker\\Documents\\GitHub\\NewsHour'

Let's open up a stata session, but this time be careful to name it. This will let us do other stuff in Stata while we are watching this program run...

In [14]:
%%stata -s nh1
cd C:\Users\mjbaker\Documents\Github\NewsHour



C:\Users\mjbaker\Documents\Github\NewsHour


In [20]:
%%stata -s nh1
use "AveragedDataDyno.dta", clear

Regenerate a marker for our game variable - which we will have to modify a bit as before! Then, get all our variables into mata and develop create a panel variable for each market in `mata`. 

In [27]:
%%stata -s nh1
capture gen game=(lnews | otherl) & lnpps!=.

In [31]:
%%stata -s nh1
mata:
    st_view(lnews=.,.,"lnews")
    st_view(nnews=.,.,"nnews")
    st_view(otherl=.,.,"otherl")
    st_view(otherc=.,.,"otherc")
    st_view(si=.,.,"si")
    st_view(mt=.,.,"mt")
    st_view(id=.,.,"stationid")
    st_view(M=.,.,"market")
    st_view(time=.,.,"timeslot")
    st_view(game=.,.,"game")
    st_view(lnpps=.,.,"lnpps")
    st_view(pop=.,.,"ACS_HH") 
    st_view(l_ACS_HH=.,.,"l_ACS_HH")
    st_view(lnewsn=.,.,"lnewsn")
    st_view(otherln=.,.,"otherln")
    st_view(nnewsn=.,.,"nnewsn")
    st_view(othercn=.,.,"othercn")
    st_view(dln=.,.,"dln")
    lnviewn=ln(pop:*si)
    m=panelsetup(M,1)
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     st_view(lnews=.,.,"lnews")

:     st_view(nnews=.,.,"nnews")

:     st_view(otherl=.,.,"otherl")

:     st_view(otherc=.,.,"otherc")

:     st_view(si=.,.,"si")

:     st_view(mt=.,.,"mt")

:     st_view(id=.,.,"stationid")

:     st_view(M=.,.,"market")

:     st_view(time=.,.,"timeslot")

:     st_view(game=.,.,"game")

:     st_view(lnpps=.,.,"lnpps")

:     st_view(pop=.,.,"ACS_HH") 

:     st_view(l_ACS_HH=.,.,"l_ACS_HH")

:     st_view(lnewsn=.,.,"lnewsn")

:     st_view(otherln=.,.,"otherln")

:     st_view(nnewsn=.,.,"nnewsn")

:     st_view(othercn=.,.,"othercn")

:     st_view(dln=.,.,"dln")

:     lnviewn=ln(pop:*si)

:     m=panelsetup(M,1)

: end


Get our previous estimation results:

In [45]:
%%stata -s nh1
mata:
    mata matuse DynoStarts23, replace
    bo[,1::4] = bo[,1::4]:^2
    mata matuse betaPDynoStarts, replace
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     mata matuse DynoStarts23, replace
(loading arate[1,27], bo[1,27], drawsbo[100,27], vals[100,1])

:     bo[,1::4] = bo[,1::4]:^2

:     mata matuse betaPDynoStarts, replace
(loading bpo[1,10], drawsbpo[200,10])

: end


Now, render the data in long form:

In [32]:
%%stata -s nh1
do MataFunctions\MataReshape.do


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:         
:         timeslots=6

:         lnewsLong=J(rows(uniqrows(id)),timeslots,.)

:         nnewsLong=J(rows(uniqrows(id)),timeslots,.)

:         otherlLong=J(rows(uniqrows(id)),timeslots,.)

:         othercLong=J(rows(uniqrows(id)),timeslots,.)

:         slnewsLong=J(rows(uniqrows(id)),timeslots,.)

:         snnewsLong=J(rows(uniqrows(id)),timeslots,.)

:         sotherlLong=J(rows(uniqrows(id)),timeslots,.)

:         sothercLong=J(rows(uniqrows(id)),timeslots,.)

:         gameLong=J(rows(uniqrows(id)),timeslots,.)

:         popLong=J(rows(uniqrows(id)),timeslots,.)  /* Not necessary but for convenience and completeness */

:         l_ACS_HHLong=J(rows(uniqrows(id)),timeslots,.)

:         
:         siLong=J(rows(uniqrows(id)),timeslots,.)

:         lnewsLongLag=J(rows(uniqrows(id)),timeslots,.)

:      

Now that that has been done, we can clean up the results from our MCMC estimation of game strategies. For one, often times our search procedure resulted in the run settling on the same strategy, but logging it as a different outcome. We therefore have to go through and pare all this down. Anyways:

In [34]:
%%stata -s nh1
mata:
    mata matuse AckerbergInfo, replace 
    mata matuse AckerbergObjs, replace
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     mata matuse AckerbergInfo, replace 
(loading Troublers[3469,6], up[16622,120], upb[16622,120], upg[16622,120], upre1[16622,120], upre1g[16622,120], upre2[16622,120], upre2g[16622,120], uv[16622,120],
 uvg[16622,120], uvre1[16622,120], uvre1g[16622,120], uvre2[16622,120], uvre2Weights[16622,6], uvre2g[16622,120], uvsg[16622,120], uvsi[16622,120],
 uvso[16622,120], vup[16622,120])

:     mata matuse AckerbergObjs, replace
(loading NashProfiles, NashProfits, NashShares, priceBounds, priceErrs, sharesBcs, simPrices)

: end


This next loop combs through all the Nash equilibria and pares them down to the number of unique profiles - so we aren't double counting results. Here goes. A first step is setting up some placeholders for everything. 

In [35]:
%%stata -s nh1
mata:
    markets=uniqrows(marketIdLong)
    indexes=asarray_keys(NashProfiles)
    draws=max(indexes[,2])

    NashProfilesNew=asarray_create("real",4)
    NashProfitsNew=asarray_create("real",3)
    NashSharesNew=asarray_create("real",3)
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     markets=uniqrows(marketIdLong)

:     indexes=asarray_keys(NashProfiles)

:     draws=max(indexes[,2])

:     NashProfitsNew=asarray_create("real",3)

:     NashSharesNew=asarray_create("real",3)

: end


## The Loop to Clean Equilibria up

In [36]:
%%stata -s nh1
mata:
    for (i=1;i<=rows(markets);i++) {
        for (d=1;d<=draws;d++) {
            gameMarkerp=select(gameLong[,1],marketIdLong:==i)
            if (colsum(gameMarkerp)>0) {
                Count=rows(uniqrows(select(indexes[,4],(indexes[,1]:==i):*(indexes[,2]:==d))))
                if (Count==1) {
                    asarray(NashProfilesNew,(i,d,1,1),asarray(NashProfiles,(i,d,1,1)))
                    asarray(NashProfilesNew,(i,d,2,1),asarray(NashProfiles,(i,d,2,1)))
                    asarray(NashProfilesNew,(i,d,3,1),asarray(NashProfiles,(i,d,3,1)))
                    asarray(NashProfitsNew,(i,d,1),asarray(NashProfits,(i,d,1)))
                    asarray(NashSharesNew,(i,d,1),asarray(NashShares,(i,d,1)))
                }
                else {
                    asarray(NashProfilesNew,(i,d,1,1),asarray(NashProfiles,(i,d,1,1)))
                    asarray(NashProfilesNew,(i,d,2,1),asarray(NashProfiles,(i,d,2,1)))
                    asarray(NashProfilesNew,(i,d,3,1),asarray(NashProfiles,(i,d,3,1)))
                    asarray(NashProfitsNew,(i,d,1),asarray(NashProfits,(i,d,1)))
                    asarray(NashSharesNew,(i,d,1),asarray(NashShares,(i,d,1)))
                    pop=2
                    tick=1
                    do {
                        noGo=0
                        for (j=1;j<=tick;j++) {
                            if (asarray(NashProfiles,(i,d,1,pop))==asarray(NashProfilesNew,(i,d,1,j))) noGo=1
                        }
                        if (noGo==0) {
                            tick++
                            asarray(NashProfilesNew,(i,d,1,tick),asarray(NashProfiles,(i,d,1,pop)))
                            asarray(NashProfilesNew,(i,d,2,tick),asarray(NashProfiles,(i,d,2,pop)))
                            asarray(NashProfilesNew,(i,d,3,tick),asarray(NashProfiles,(i,d,3,pop)))
                            asarray(NashProfitsNew,(i,d,tick),asarray(NashProfits,(i,d,pop)))
                            asarray(NashSharesNew,(i,d,tick),asarray(NashShares,(i,d,pop)))
                        }
                        pop++
                    } while (pop<=Count)
                }
            }
        }
    }   
end    
    
    
    


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     for (i=1;i<=rows(markets);i++) {
>         for (d=1;d<=draws;d++) {
>             gameMarkerp=select(gameLong[,1],marketIdLong:==i)
>             if (colsum(gameMarkerp)>0) {
>                 Count=rows(uniqrows(select(indexes[,4],(indexes[,1]:==i):*(indexes[,2]:==d))))
>                 if (Count==1) {
>                     asarray(NashProfilesNew,(i,d,1,1),asarray(NashProfiles,(i,d,1,1)))
>                     asarray(NashProfilesNew,(i,d,2,1),asarray(NashProfiles,(i,d,2,1)))
>                     asarray(NashProfilesNew,(i,d,3,1),asarray(NashProfiles,(i,d,3,1)))
>                     asarray(NashProfitsNew,(i,d,1),asarray(NashProfits,(i,d,1)))
>                     asarray(NashSharesNew,(i,d,1),asarray(NashShares,(i,d,1)))
>                 }
>                 else {
>                     asarray(NashProfilesNew

We now have everything we need in the associative arrays affixed with `New`, so we can drop the others to save space:

In [37]:
%%stata -s nh1
mata:
    mata drop NashProfiles
    mata drop NashProfits
    mata drop NashShares
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     mata drop NashProfiles

:     mata drop NashProfits

:     mata drop NashShares

: end


## Reshaping and packing up all this information...
The next thing we will do is change our revised equilibrium information into a packable and useable form, and save it all so it can be used easily in estimation. First part: reshaping our AKW error terms:

In [38]:
%%stata -s nh1
mata:
    Up=J(rows(up)*timeslots,0,.)
    Upg=J(rows(up)*timeslots,0,.)
    Upb=J(rows(up)*timeslots,0,.)
    Uv=J(rows(up)*timeslots,0,.)
    Uvg=J(rows(up)*timeslots,0,.)
    Vup=J(rows(up)*timeslots,0,.)
    Uvsi=J(rows(up)*timeslots,0,.)
    Uvsg=J(rows(up)*timeslots,0,.)
    Uvso=J(rows(up)*timeslots,0,.)
    Uvre1=J(rows(up)*timeslots,0,.)
    Uvre2=J(rows(up)*timeslots,0,.)
    Upre1=J(rows(up)*timeslots,0,.)
    Upre2=J(rows(up)*timeslots,0,.)
    Uvre1g=J(rows(up)*timeslots,0,.)
    Uvre2g=J(rows(up)*timeslots,0,.)
    Upre1g=J(rows(up)*timeslots,0,.)
    Upre2g=J(rows(up)*timeslots,0,.)

    counter=1
    for (d=1;d<=draws;d++) {
        Up=Up,colshape(up[,counter::counter+timeslots-1],1)
        Upg=Upg,colshape(upg[,counter::counter+timeslots-1],1)
        Upb=Upb,colshape(upb[,counter::counter+timeslots-1],1)
        Uv=Uv,colshape(uv[,counter::counter+timeslots-1],1)
        Uvg=Uvg,colshape(uvg[,counter::counter+timeslots-1],1)
        Vup=Vup,colshape(vup[,counter::counter+timeslots-1],1)
        Uvsi=Uvsi,colshape(uvsi[,counter::counter+timeslots-1],1)
        Uvsg=Uvsg,colshape(uvsg[,counter::counter+timeslots-1],1)
        Uvso=Uvso,colshape(uvso[,counter::counter+timeslots-1],1)
        Uvre1=Uvre1,colshape(uvre1[,counter::counter+timeslots-1],1)
        Uvre2=Uvre2,colshape(uvre2[,counter::counter+timeslots-1],1)
        Upre1=Upre1,colshape(upre1[,counter::counter+timeslots-1],1)
        Upre2=Upre2,colshape(upre2[,counter::counter+timeslots-1],1)
        Uvre1g=Uvre1g,colshape(uvre1g[,counter::counter+timeslots-1],1)
        Uvre2g=Uvre2g,colshape(uvre2g[,counter::counter+timeslots-1],1)
        Upre1g=Upre1g,colshape(upre1g[,counter::counter+timeslots-1],1)
        Upre2g=Upre2g,colshape(upre2g[,counter::counter+timeslots-1],1)
        
        counter=counter+timeslots
    }
end



------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     Up=J(rows(up)*timeslots,0,.)

:     Upg=J(rows(up)*timeslots,0,.)

:     Upb=J(rows(up)*timeslots,0,.)

:     Uv=J(rows(up)*timeslots,0,.)

:     Uvg=J(rows(up)*timeslots,0,.)

:     Vup=J(rows(up)*timeslots,0,.)

:     Uvsi=J(rows(up)*timeslots,0,.)

:     Uvsg=J(rows(up)*timeslots,0,.)

:     Uvso=J(rows(up)*timeslots,0,.)

:     Uvre1=J(rows(up)*timeslots,0,.)

:     Uvre2=J(rows(up)*timeslots,0,.)

:     Upre1=J(rows(up)*timeslots,0,.)

:     Upre2=J(rows(up)*timeslots,0,.)

:     Uvre1g=J(rows(up)*timeslots,0,.)

:     Uvre2g=J(rows(up)*timeslots,0,.)

:     Upre1g=J(rows(up)*timeslots,0,.)

:     Upre2g=J(rows(up)*timeslots,0,.)

:     for (d=1;d<=draws;d++) {
>         Up=Up,colshape(up[,counter::counter+timeslots-1],1)
>         Upg=Upg,colshape(upg[,counter::counter+timeslots-1],1)
>         Upb=Upb,colshap

Here we will pull out all of the upper bounds on error terms, so we can turn them into sampling weights based on truncated normal variables, in a way that conforms with everything else. Anyways

In [46]:
%%stata -s nh1
mata:
    Keys=uniqrows(asarray_keys(priceErrs)[,1])
    stationid=statIdLong#J(6,1,1)
    sdmodp=exp(bpo[9])
    
    ErrsToInt=J(rows(stationid),draws,.)
    ErrBounds=J(rows(stationid),draws,.)
    for (i=1;i<=rows(Keys);i++) {
        for (d=1;d<=draws;d++) {
            errToPut=asarray(priceErrs,(Keys[i],d))[1,]
            bouToPut=asarray(priceBounds,(Keys[i],d))[1,]
            minPos=min(mm_which(bouToPut:!=.))
            errToPut=errToPut[minPos]
            bouToPut=bouToPut[minPos]
            place=max(mm_which(stationid:==Keys[i]))
            ErrsToInt[place,d]=errToPut
            ErrBounds[place,d]=bouToPut
        }
    }    
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     Keys=uniqrows(asarray_keys(priceErrs)[,1])

:     stationid=statIdLong#J(6,1,1)

:     sdmodp=exp(bpo[9])

:     
:     ErrsToInt=J(rows(stationid),draws,.)

:     ErrBounds=J(rows(stationid),draws,.)

:     for (i=1;i<=rows(Keys);i++) {
>         for (d=1;d<=draws;d++) {
>             errToPut=asarray(priceErrs,(Keys[i],d))[1,]
>             bouToPut=asarray(priceBounds,(Keys[i],d))[1,]
>             minPos=min(mm_which(bouToPut:!=.))
>             errToPut=errToPut[minPos]
>             bouToPut=bouToPut[minPos]
>             place=max(mm_which(stationid:==Keys[i]))
>             ErrsToInt[place,d]=errToPut
>             ErrBounds[place,d]=bouToPut
>         }
>     }    

: end


This next piece of code combs through our Nash equilibria (revised) and renders sampling weights. Note the first step of getting the keys from the associative array so we can loop over them (which marks our games)

In [47]:
%%stata -s nh1
mata:
    lnE=ln(normal(ErrBounds/sdmodp))
    Keys=asarray_keys(NashProfilesNew)
    market=marketIdLong#J(timeslots,1,1)
    gameMarkId=uniqrows(Keys[,1])
    NashW=J(rows(market),draws,.)
    
    for (i=1;i<=rows(gameMarkId);i++) {
        WtoAdd=J(1,draws,.)
        for (d=1;d<=draws;d++) {
            WtoAdd[d]=rows(select(Keys[,4],(Keys[,1]:==gameMarkId[i]):*(Keys[,2]:==d):*(Keys[,3]:==1)))
        }
        maxPos=max(mm_which(market:==gameMarkId[i]))
        NashW[maxPos,]=ln(1:/WtoAdd)
    }
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     lnE=ln(normal(ErrBounds/sdmodp))

:     Keys=asarray_keys(NashProfilesNew)

:     market=marketIdLong#J(timeslots,1,1)

:     gameMarkId=uniqrows(Keys[,1])

:     NashW=J(rows(market),draws,.)

:     
:     for (i=1;i<=rows(gameMarkId);i++) {
>         WtoAdd=J(1,draws,.)
>         for (d=1;d<=draws;d++) {
>             WtoAdd[d]=rows(select(Keys[,4],(Keys[,1]:==gameMarkId[i]):*(Keys[,2]:==d):*(Keys[,3]:==1)))
>         }
>         maxPos=max(mm_which(market:==gameMarkId[i]))
>         NashW[maxPos,]=ln(1:/WtoAdd)
>     }

: end


Make into sampling weights. We now make some sampling weights

In [49]:
%%stata -s nh1
mata:
    weights1=1:/rowsum(gameLong)#J(1,6,1)
    weights1=colshape(weights1,1)

    mLong=panelsetup(marketIdLong,1)
    weights2=J(rows(statIdLong),timeslots,.)
    for (i=1;i<=rows(mLong);i++) {
        gP=panelsubmatrix(gameLong,i,mLong)
        gPtotal=colsum(gP)
        weightsP=(gPtotal:*gP)
    weights2[mLong[i,1]::mLong[i,2],.]=weightsP
    }
    _editvalue(weights2,0,.)
    weights2=colshape(1:/weights2,1)
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     weights1=1:/rowsum(gameLong)#J(1,6,1)

:     weights1=colshape(weights1,1)

:     weights2=J(rows(statIdLong),timeslots,.)

:     for (i=1;i<=rows(mLong);i++) {
>         gP=panelsubmatrix(gameLong,i,mLong)
>         gPtotal=colsum(gP)
>         weightsP=(gPtotal:*gP)
>     weights2[mLong[i,1]::mLong[i,2],.]=weightsP
>     }

:     _editvalue(weights2,0,.)

:     weights2=colshape(1:/weights2,1)

: end


Having collated everything, we now get put glob it all and save it for use in actual estimation! I really have to learn how to use macros and pass them around...

In [50]:
%%stata -s nh1
mata:
    mata matsave gsAndus NashW Up Upg Upb Uv Uvg Uvsi Uvsg Uvso Uvre1 Uvre2 Upre1 Upre2 Uvre1g Uvre2g Upre1g Upre2g Vup market id mt weights1 weights2, replace
end


------------------------------------------------- mata (type end to exit) -------------------------------------------------------------------------------------------
:     mata matsave gsAndus NashW Up Upg Upb Uv Uvg Uvsi Uvsg Uvso Uvre1 Uvre2 Upre1 Upre2 Uvre1g Uvre2g Upre1g Upre2g Vup market id mt weights1 weights2, replace
(saving NashW[99732,20], Up[99732,20], Upb[99732,20], Upg[99732,20], Upre1[99732,20], Upre1g[99732,20], Upre2[99732,20], Upre2g[99732,20], Uv[99732,20],
 Uvg[99732,20], Uvre1[99732,20], Uvre1g[99732,20], Uvre2[99732,20], Uvre2g[99732,20], Uvsg[99732,20], Uvsi[99732,20], Uvso[99732,20], Vup[99732,20], id[99732,1],
 market[99732,1], mt[99732,1], weights1[99732,1], weights2[99732,1])
file gsAndus.mmat saved

: end


Close down the session

In [51]:
%%stata
close all

Terminated 2 running Stata processes
